In [ ]:
# 📦 Installer les dépendances
!pip install pydub transformers torchaudio

# 🔊 Chargement et re-échantillonnage
import torchaudio
import torchaudio.transforms as T

# 🧠 Transcription avec Wav2Vec2.0
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
!apt-get install ffmpeg -y

In [ ]:
from google.colab import files
from pydub import AudioSegment
import torchaudio
import torchaudio.transforms as T
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import os

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # 🔄 Conversion M4A → WAV
  filename = fn
  audio = AudioSegment.from_file(filename, format="m4a")
  audio.export("converted.wav", format="wav")

  # 🔊 Chargement et re-échantillonnage
  waveform, sample_rate = torchaudio.load("converted.wav")
  resampler = T.Resample(orig_freq=sample_rate, new_freq=16000)
  waveform_resampled = resampler(waveform)

  # 🧠 Transcription avec Wav2Vec2.0
  processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
  model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

  input_values = processor(waveform_resampled.squeeze(), sampling_rate=16000, return_tensors="pt").input_values
  logits = model(input_values).logits
  predicted_ids = torch.argmax(logits, dim=-1)

  transcription = processor.decode(predicted_ids[0])
  print("📝 Transcription :", transcription)